In [ ]:
%load_ext blackcellmagic
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import sys
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Define 'viral' and 'non-viral' groups:

In [ ]:
# Create the 'all-viral_label' column
data_date = "XXX"
data_path = f"XXX"
data_df = pd.read_csv(data_path, low_memory=False)

# Drop the 'all-viral_label' column if it already exists
if 'all-viral_label' in data_df.columns:
    data_df = data_df.drop(columns=['all-viral_label'])

# Exclude non-viral diagnoses from viral label calculation, but check for confirmed malaria or syphilis
exclude_labels = {"Malaria", "Syphilis", "HIV"}  # Exclude bacterial and chronic viral infections
label_cols = [
    col for col in data_df.columns
    if col.endswith('_label') and not any(exclude in col for exclude in exclude_labels)
]
malaria_label_col = "Malaria_label"
syphilis_label_col = "Syphilis_label"
hiv_label_col = "HIV_label"

# Find negative samples with as many confirmed negative tests as possible
no_positive_mask = ~((data_df[[col for col in data_df.columns if col.endswith('_label')]] == 1).any(axis=1))
data_df_no_positives = data_df[no_positive_mask]
num_negatives_per_sample = (data_df_no_positives[label_cols] == 0).sum(axis=1).sort_values(ascending=False)
# # Find the minimum number of labels==0 results that still allows us to keep ~1000 negative samples
# desired_n = 1000
# # Print the distribution of negative test counts for debugging
# print("Distribution of negative test counts among samples with no positive viral labels:")
# print(num_negatives_per_sample.value_counts().sort_index(ascending=False))
# if len(num_negatives_per_sample) < desired_n:
#     print(f"Warning: Only {len(num_negatives_per_sample)} samples with no positive viral labels, fewer than desired {desired_n}.")
#     negative_test_threshold = num_negatives_per_sample.min()
# else:
#     negative_test_threshold = num_negatives_per_sample.iloc[desired_n - 1]
# print(f"Minimum number of negative tests to keep ~{desired_n} samples: {negative_test_threshold}")
# Define negative_test_threshold manually instead
negative_test_threshold = 5
# Get indices of high-confidence negatives (most negative viral tests)
high_conf_neg_indices = num_negatives_per_sample[num_negatives_per_sample >= negative_test_threshold].index
num_selected = (num_negatives_per_sample >= negative_test_threshold).sum()
print(f"Number of samples with >= {negative_test_threshold} negative tests: {num_selected}")

# Definitions:
# Viral: Any confirmed viral diagnosis (including all co-infections)
# Non-viral: Top N samples with the most confirmed negative tests as negative samples + N malaria-confirmed (no co-infections) samples

# Randomly select indices of malaria-confirmed samples (no co-infections) to use as negative samples
malaria_confirmed_mask = (
    (data_df[malaria_label_col] == 1) &
    ~((data_df[[col for col in data_df.columns if col.endswith('_label') and col != malaria_label_col]] == 1).any(axis=1))
)
malaria_num = num_selected
malaria_confirmed_indices = data_df[malaria_confirmed_mask].sample(n=min(malaria_num, malaria_confirmed_mask.sum()), random_state=42).index

def compute_all_viral_label(row):
    labels = row[label_cols]
    idx = row.name  # row index

    # Positive samples have at least one confirmed viral diagnosis (inclusive of co-infections)
    if (labels == 1).any():
        return 1

    # Use the top N samples with the most confirmed negative tests
    elif idx in high_conf_neg_indices:
        return 0

    # Use some malaria-confirmed samples as negative
    elif idx in malaria_confirmed_indices:
        return 0

    else:
        return 2

data_df['all-viral_label'] = data_df.apply(compute_all_viral_label, axis=1)

# Save data frame with new 'all-viral_label' column
data_df.to_csv(data_path, index=False)

print(data_df["all-viral_label"].value_counts())
data_df.head()

In [ ]:
# This data does not include CARMEN results
data_df[data_df["Ebola_label"]==1]